# Reading Dataset , Cleaning, Preprocessing

In [1]:
import pandas as pd
import re
import nltk
import numpy as np
from collections import Counter
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import math  # Import the math module for perplexity calculation
import matplotlib.pyplot as plt 
import torch.nn.functional as F

# Replace 'train.csv' and 'test.csv' with the actual file paths to your CSV files.
train_csv_file = 'train.csv'
test_csv_file = 'test.csv'

# Read the training data from train.csv
train_data = pd.read_csv(train_csv_file)

# Read the test data from test.csv
test_data = pd.read_csv(test_csv_file)

# Split the dataset into train and development sets.
dev_set_size = 7600
train_set = train_data.iloc[dev_set_size:]  # The portion after the development set.
dev_set = train_data.iloc[:dev_set_size]    # The first 7600 entries as the development set.

# Extract sentences and labels for train, dev, and test sets.
train_sentences = train_set['Description'].tolist()
train_labels = train_set['Class Index'].tolist()

dev_sentences = dev_set['Description'].tolist()
dev_labels = dev_set['Class Index'].tolist()

test_sentences = test_data['Description'].tolist()
test_labels = test_data['Class Index'].tolist()

# Function to clean a sentence
def clean_sentence(sentence):
    # Convert to lowercase
    sentence = sentence.lower()
    # Remove special characters, numbers, and extra whitespace
    sentence = re.sub(r'[^a-zA-Z\s]', '', sentence)
    # Tokenize the sentence (split into words)
    tokens = sentence.split()
    return ' '.join(tokens)

# Clean the sentences in the train, dev, and test sets
train_sentences_cleaned = [clean_sentence(sentence) for sentence in train_sentences]
dev_sentences_cleaned = [clean_sentence(sentence) for sentence in dev_sentences]
test_sentences_cleaned = [clean_sentence(sentence) for sentence in test_sentences]

# Tokenize the sentences using NLTK
nltk.download('punkt')
from nltk.tokenize import word_tokenize

train_sentences_tokenized = [word_tokenize(sentence) for sentence in train_sentences_cleaned]
dev_sentences_tokenized = [word_tokenize(sentence) for sentence in dev_sentences_cleaned]
test_sentences_tokenized = [word_tokenize(sentence) for sentence in test_sentences_cleaned]


[nltk_data] Downloading package punkt to /home/yash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Char Preprocessing (For ELMO)

In [2]:
train_char_sentences = [[list(word) for word in sentence] for sentence in train_sentences_tokenized]
dev_char_sentences = [[list(word) for word in sentence] for sentence in dev_sentences_tokenized]
test_char_sentences = [[list(word) for word in sentence] for sentence in test_sentences_tokenized]

In [3]:
# Define the desired sequence lengths
max_sentence_length = 128
max_word_length = 64  # Or your desired maximum word length

# Create character-level vocabulary and assign indices
char_to_idx = {'<PAD>': 0, '<UNK>': 1}  # Add padding and unknown tokens
char_vocab_size = len(char_to_idx)
char_embedding_dim = 50  # Set an appropriate embedding dimension

# Function to pad or truncate a list to a fixed length
def pad_or_truncate(lst, length, padding_value):
    if len(lst) >= length:
        return lst[:length]
    else:
        return lst + [padding_value] * (length - len(lst))

# Convert characters to indices for training data
char_indices_train = [
    [
        pad_or_truncate(
            [char_to_idx.get(char, 1) for char in word],
            max_word_length,
            0
        )
        for word in sentence
    ]
    for sentence in train_char_sentences
]

# Convert characters to indices for development data
char_indices_dev = [
    [
        pad_or_truncate(
            [char_to_idx.get(char, 1) for char in word],
            max_word_length,
            0
        )
        for word in sentence
    ]
    for sentence in dev_char_sentences
]

# Convert characters to indices for test data
char_indices_test = [
    [
        pad_or_truncate(
            [char_to_idx.get(char, 1) for char in word],
            max_word_length,
            0
        )
        for word in sentence
    ]
    for sentence in test_char_sentences
]

# Pad or truncate character sequences for training data
char_indices_train = [
    pad_or_truncate(sentence, max_sentence_length, [0] * max_word_length)
    for sentence in char_indices_train
]

# Pad or truncate character sequences for development data
char_indices_dev = [
    pad_or_truncate(sentence, max_sentence_length, [0] * max_word_length)
    for sentence in char_indices_dev
]

# Pad or truncate character sequences for test data
char_indices_test = [
    pad_or_truncate(sentence, max_sentence_length, [0] * max_word_length)
    for sentence in char_indices_test
]

# Convert to PyTorch tensors
char_input_data_train = torch.tensor(char_indices_train)
char_input_data_dev = torch.tensor(char_indices_dev)
char_input_data_test = torch.tensor(char_indices_test)


# Create DataLoader instances
batch_size = 32

train_loader_elmo = DataLoader(TensorDataset(char_input_data_train[:10000]), batch_size=batch_size, shuffle=False)
dev_loader_elmo = DataLoader(TensorDataset(char_input_data_dev[:500]), batch_size=batch_size, shuffle=False)
test_loader_elmo = DataLoader(TensorDataset(char_input_data_test[:500]), batch_size=batch_size, shuffle=False)



In [4]:
print(char_input_data_train.shape)
print(char_input_data_dev.shape)
print(char_input_data_test.shape)

torch.Size([112400, 128, 64])
torch.Size([7600, 128, 64])
torch.Size([7600, 128, 64])


# Char Preprocessing (For Language Modelling)

In [5]:
# Convert it to a list of words where each word is a list of characters
train_char_sentences_flattened = [char_list for word_list in train_char_sentences for char_list in word_list]
dev_char_sentences_flattened = [char_list for word_list in dev_char_sentences for char_list in word_list]
test_char_sentences_flattened = [char_list for word_list in test_char_sentences for char_list in word_list]

In [6]:
# Convert characters to indices for training data
char_indices_train_flattened = [
    pad_or_truncate(
        [char_to_idx.get(char, 1) for char in word],
        max_word_length,
        0
    )
    for sentence in train_char_sentences_flattened
    for word in sentence
]

# Convert characters to indices for development data
char_indices_dev_flattened = [
    pad_or_truncate(
        [char_to_idx.get(char, 1) for char in word],
        max_word_length,
        0
    )
    for sentence in dev_char_sentences_flattened
    for word in sentence
]

# Convert characters to indices for test data
char_indices_test_flattened = [
    pad_or_truncate(
        [char_to_idx.get(char, 1) for char in word],
        max_word_length,
        0
    )
    for sentence in test_char_sentences_flattened
    for word in sentence
]

In [7]:
# Convert to PyTorch tensors
char_input_data_train_flattened = torch.tensor(char_indices_train_flattened)
char_input_data_dev_flattened = torch.tensor(char_indices_dev_flattened)
char_input_data_test_flattened = torch.tensor(char_indices_test_flattened)

In [8]:
print(char_input_data_train_flattened.shape)
print(char_input_data_dev_flattened.shape)
print(char_input_data_test_flattened.shape)

torch.Size([17307370, 64])
torch.Size([1213019, 64])
torch.Size([1167836, 64])


In [9]:
# Create DataLoader instances
batch_size = 32

train_loader = DataLoader(TensorDataset(char_input_data_train_flattened[:10000]), batch_size=batch_size, shuffle=False)
dev_loader = DataLoader(TensorDataset(char_input_data_dev_flattened[:500]), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(char_input_data_test_flattened[:500]), batch_size=batch_size, shuffle=False)

In [10]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [29]:
class CharCNN(nn.Module):
    def __init__(self, char_vocab_size, char_embedding_dim, num_filters, kernel_sizes):
        super(CharCNN, self).__init__()
        self.char_embedding = nn.Embedding(char_vocab_size, char_embedding_dim)
        self.conv_layers = nn.ModuleList([
            nn.Conv1d(char_embedding_dim, num_filters, kernel_size) for kernel_size in kernel_sizes
        ])

    def forward(self, char_input):
        char_embedded = self.char_embedding(char_input)  # [batch_size, max_word_len, char_embedding_dim]
        char_embedded = char_embedded.permute(0, 2, 1)  # [batch_size, char_embedding_dim, max_word_len]

        conv_outputs = [conv(char_embedded) for conv in self.conv_layers]  # Apply convolutional layers
        pooled_outputs = [torch.max(conv_output, dim=2)[0] for conv_output in conv_outputs]  # Max-pooling
        char_cnn_output = torch.cat(pooled_outputs, dim=1)  # Concatenate pooled outputs
        return char_cnn_output
    

class ELMo(nn.Module):
    def __init__(self, hidden_dim, batch_size, max_len, char_cnn1=None, char_cnn2=None, num_classes=4):
        super(ELMo, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.max_len = max_len
        self.num_classes = num_classes

        # Character CNN Embedding
        self.char_cnn1 = char_cnn1
        self.char_cnn1 = char_cnn2

        # Forward Language Model
        self.lstm_forward1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.lstm_forward2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)

        # Backward Language Model
        self.lstm_backward1 = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.lstm_backward2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)

        # Linear layer for mode 1
        self.linear_mode1 = nn.Linear(200, vocab_size)

        # Linear layer for mode 2
        self.linear_mode2 = nn.Linear(200, vocab_size)

        # Define trainable weights for mode 0 as a vector of size 3 with values 0.33 each
        self.weights_mode0 = nn.Parameter(torch.tensor([0.33, 0.33, 0.33], dtype=torch.float32))

        self.final_linear = nn.Linear(self.max_len * self.embedding_dim * 2, num_classes)

    def forward(self, input_data, mode):
        if mode == 0:

            char_input = input_data  # Input is now character-level data

            # Use char_cnn_output for further processing
            forward_embed = self.char_cnn1(char_input)

            # Use char_cnn_output for further processing
            backward_embed = self.char_cnn2(char_input)


            # Forward Language Model 1
            forward_lstm1, _ = self.lstm_forward1(forward_embed)
            
            # Backward Language Model 1
            backward_lstm1, _ = self.lstm_backward1(backward_embed)
            
            # Concatenate forward1 and backward1 outputs
            lstm_concat1 = torch.cat((forward_lstm1, backward_lstm1), dim=-1)
            
            # Forward Language Model 2
            forward_lstm2, _ = self.lstm_forward2(forward_lstm1)
            
            # Backward Language Model 2
            backward_lstm2, _ = self.lstm_backward2(backward_lstm1)
            
            # Concatenate forward2 and backward2 outputs
            lstm_concat2 = torch.cat((forward_lstm2, backward_lstm2), dim=-1)

            # Concatenate forward and backward embeddings word by word
            concatenated_embeddings = torch.cat((forward_embed, backward_embed), dim=-1)
            
            # Apply trainable weights for mode 0 to the concatenated tensors
            weighted_embeddings = concatenated_embeddings * self.weights_mode0[0]
            weighted_lstm1 = lstm_concat1 * self.weights_mode0[1]
            weighted_lstm2 = lstm_concat2 * self.weights_mode0[2]

            # Sum along the last dimension (600) to get a (32 x 256 x 200) tensor
            summed_tensor = weighted_embeddings + weighted_lstm1 + weighted_lstm2

            # Reshape the output to 32 x (256 x vocabSize)
            output = summed_tensor.view(-1, self.max_len * self.embedding_dim *2)

            # Additional linear layer to reduce the output to 32 x 4
            output = self.final_linear(output)

            # output = F.softmax(output, dim=1)

            return output

        elif mode == 1:
            char_input = input_data  # Input is now character-level data

            # Use char_cnn_output for further processing
            forward_embed = self.char_cnn1(char_input)


            # Forward Language Model 1
            forward_lstm1, _ = self.lstm_forward1(forward_embed, None) # Set batch_first to False

            # Forward Language Model 2
            forward_lstm2, _ = self.lstm_forward2(forward_lstm1, None) # Set batch_first to False

            # Concatenate forward_lstm1 and forward_lstm2 outputs
            lstm_concat = torch.cat((forward_lstm1, forward_lstm2), dim=-1)

            # Apply linear layer for mode 1
            output = self.linear_mode1(lstm_concat)

            return output
        
        elif mode == 2:
            char_input = input_data  # Input is now character-level data

            # Use char_cnn_output for further processing
            backward_embed = self.char_cnn2(char_input)
            
            # Backward Language Model 1
            backward_lstm1, _ = self.lstm_backward1(backward_embed, None) # Set batch_first to False

            # Backward Language Model 2
            backward_lstm2, _ = self.lstm_backward2(backward_lstm1, None) # Set batch_first to False

            # Concatenate backward_lstm1 and backward_lstm2 outputs
            lstm_concat = torch.cat((backward_lstm1, backward_lstm2), dim=-1)

            # Apply linear layer for mode 2
            output = self.linear_mode2(lstm_concat)

            return output


In [30]:
vocab_size = len(vocab)
embedding_dim = 100
hidden_dim = 100
max_len = 256
num_classes = 4

num_filters = 100
kernel_sizes = [3, 4, 5]  # You can adjust these values based on experimentation

# Define the character-level CNN
char_cnn1 = CharCNN(char_vocab_size, char_embedding_dim, num_filters, kernel_sizes)
char_cnn2 = CharCNN(char_vocab_size, char_embedding_dim, num_filters, kernel_sizes)

# Define the ELMo model
elmo = ELMo(vocab_size, embedding_dim, hidden_dim, batch_size, max_len, embedding_matrix, char_cnn1=char_cnn1, char_cnn2=char_cnn2, use_char_cnn=True, num_classes=num_classes)

# Move the model to the appropriate device (e.g., GPU)
elmo.to(device)

ELMo(
  (embedding1): Embedding(81098, 100)
  (embedding2): Embedding(81098, 100)
  (char_cnn1): CharCNN(
    (char_embedding): Embedding(2, 50)
    (conv_layers): ModuleList(
      (0): Conv1d(50, 100, kernel_size=(3,), stride=(1,))
      (1): Conv1d(50, 100, kernel_size=(4,), stride=(1,))
      (2): Conv1d(50, 100, kernel_size=(5,), stride=(1,))
    )
  )
  (lstm_forward1): LSTM(100, 100, batch_first=True)
  (lstm_forward2): LSTM(100, 100, batch_first=True)
  (lstm_backward1): LSTM(100, 100, batch_first=True)
  (lstm_backward2): LSTM(100, 100, batch_first=True)
  (linear_mode1): Linear(in_features=200, out_features=81098, bias=True)
  (linear_mode2): Linear(in_features=200, out_features=81098, bias=True)
  (final_linear): Linear(in_features=51200, out_features=4, bias=True)
)

In [ ]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(elmo.parameters(), lr=0.001)  # Adjust the learning rate as needed

# Training loop for mode 1 (next word prediction)
def train_mode1(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    total_tokens = 0  # Total tokens processed in the dataset

    for inputs in tqdm(train_loader):
        inputs = inputs[0]
        inputs = inputs.to(device)
        optimizer.zero_grad()

        # Use the first 5 words as input and the last 5 words as expected output
        input_seq = inputs[:, :5]
        target_seq = inputs[:, 1:]

        # Forward pass
        outputs = model(input_seq, mode=1)

        # Calculate the loss
        loss = criterion(outputs.permute(0, 2, 1), target_seq)  # Permute for CrossEntropyLoss
        total_loss += loss.item()

        # Calculate the total tokens processed
        total_tokens += target_seq.numel()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Calculate perplexity based on the total loss and tokens
    perplexity = math.exp(total_loss / total_tokens)

    avg_loss = total_loss / total_tokens

    return avg_loss

# Development loop
def eval_mode1(model, dev_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_tokens = 0  # Total tokens processed in the dataset

    with torch.no_grad():
        for inputs in tqdm(dev_loader):
            inputs = inputs[0]
            inputs = inputs.to(device)

            # Use the first 5 words as input and the last 5 words as expected output
            input_seq = inputs[:, :5]
            target_seq = inputs[:, 1:]

            # Forward pass
            outputs = model(input_seq, mode=1)

            # Calculate the loss
            loss = criterion(outputs.permute(0, 2, 1), target_seq)  # Permute for CrossEntropyLoss
            total_loss += loss.item()

            # Calculate the total tokens processed
            total_tokens += target_seq.numel()

    avg_loss = total_loss / total_tokens

    return avg_loss

# Testing loop
def test_mode1(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_tokens = 0  # Total tokens processed in the dataset

    with torch.no_grad():
        for inputs in tqdm(test_loader):
            inputs = inputs[0]
            inputs = inputs.to(device)

            # Use the first 5 words as input and the last 5 words as expected output
            input_seq = inputs[:, :5]
            target_seq = inputs[:, 1:]

            # Forward pass
            outputs = model(input_seq, mode=1)

            # Calculate the loss
            loss = criterion(outputs.permute(0, 2, 1), target_seq)  # Permute for CrossEntropyLoss
            total_loss += loss.item()

            # Calculate the total tokens processed
            total_tokens += target_seq.numel()

    avg_loss = total_loss / total_tokens

    return avg_loss

# Training loop for mode 2 (previous word prediction)
def train_mode2(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    total_tokens = 0  # Total tokens processed in the dataset

    for inputs in tqdm(train_loader):
        inputs = inputs[0]
        inputs = inputs.to(device)
        optimizer.zero_grad()

        # Use the first 5 words as input and the last 5 words as expected output
        input_seq = inputs[:, :5]
        target_seq = inputs[:, 1:]

        # Forward pass
        outputs = model(input_seq, mode=2)

        # Calculate the loss
        loss = criterion(outputs.permute(0, 2, 1), target_seq)  # Permute for CrossEntropyLoss
        total_loss += loss.item()

        # Calculate the total tokens processed
        total_tokens += target_seq.numel()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / total_tokens

    return avg_loss

# Development loop
def eval_mode2(model, dev_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_tokens = 0  # Total tokens processed in the dataset

    with torch.no_grad():
        for inputs in tqdm(dev_loader):
            inputs = inputs[0]
            inputs = inputs.to(device)

            # Use the first 5 words as input and the last 5 words as expected output
            input_seq = inputs[:, :5]
            target_seq = inputs[:, 1:]

            # Forward pass
            outputs = model(input_seq, mode=2)

            # Calculate the loss
            loss = criterion(outputs.permute(0, 2, 1), target_seq)  # Permute for CrossEntropyLoss
            total_loss += loss.item()

            # Calculate the total tokens processed
            total_tokens += target_seq.numel()

    avg_loss = total_loss / total_tokens

    return avg_loss

# Testing loop
def test_mode2(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_tokens = 0  # Total tokens processed in the dataset

    with torch.no_grad():
        for inputs in tqdm(test_loader):
            inputs = inputs[0]
            inputs = inputs.to(device)

            # Use the first 5 words as input and the last 5 words as expected output
            input_seq = inputs[:, :5]
            target_seq = inputs[:, 1:]

            # Forward pass
            outputs = model(input_seq, mode=2)

            # Calculate the loss
            loss = criterion(outputs.permute(0, 2, 1), target_seq)  # Permute for CrossEntropyLoss
            total_loss += loss.item()

            # Calculate the total tokens processed
            total_tokens += target_seq.numel()

    avg_loss = total_loss / total_tokens

    return avg_loss


In [ ]:
# Training hyperparameters
num_epochs = 5  # Adjust as needed

# Initialize empty lists to store loss and perplexity values
loss_mode1_values = []
perplexity_mode1_values = []
loss_mode2_values = []
perplexity_mode2_values = []
dev_loss_mode1_values = []  # To store dev loss
dev_perplexity_mode1_values = []  # To store dev perplexity
dev_loss_mode2_values = []  # To store dev loss
dev_perplexity_mode2_values = []  # To store dev perplexity
test_loss_mode1_values = []  # To store test loss
test_perplexity_mode1_values = []  # To store test perplexity
test_loss_mode2_values = []  # To store test loss
test_perplexity_mode2_values = []  # To store test perplexity

# Training loop for mode 1
for epoch in range(num_epochs):
    train_loss_mode1 = train_mode1(elmo, train_loader, optimizer, criterion, device)
    loss_mode1_values.append(train_loss_mode1)

    # Calculate perplexity based on the loss (assuming cross-entropy loss)
    perplexity_mode1 = math.exp(train_loss_mode1)
    perplexity_mode1_values.append(perplexity_mode1)

    print(f"Epoch {epoch+1}/{num_epochs} (Mode 1) - Train Loss: {train_loss_mode1:.4f}, Train Perplexity: {perplexity_mode1:.4f}")

    # Evaluate on dev dataset
    dev_loss_mode1 = eval_mode1(elmo, dev_loader, criterion, device)
    dev_loss_mode1_values.append(dev_loss_mode1)

    # Calculate perplexity based on the dev loss (assuming cross-entropy loss)
    dev_perplexity_mode1 = math.exp(dev_loss_mode1)
    dev_perplexity_mode1_values.append(dev_perplexity_mode1)

    print(f"Epoch {epoch+1}/{num_epochs} (Mode 1) - Dev Loss: {dev_loss_mode1:.4f}, Dev Perplexity: {dev_perplexity_mode1:.4f}")

# Training loop for mode 2
for epoch in range(num_epochs):
    train_loss_mode2 = train_mode2(elmo, train_loader, optimizer, criterion, device)
    loss_mode2_values.append(train_loss_mode2)

    # Calculate perplexity based on the loss (assuming cross-entropy loss)
    perplexity_mode2 = math.exp(train_loss_mode2)
    perplexity_mode2_values.append(perplexity_mode2)

    print(f"Epoch {epoch+1}/{num_epochs} (Mode 2) - Train Loss: {train_loss_mode2:.4f}, Train Perplexity: {perplexity_mode2:.4f}")

    # Evaluate on dev dataset
    dev_loss_mode2 = eval_mode2(elmo, dev_loader, criterion, device)
    dev_loss_mode2_values.append(dev_loss_mode2)

    # Calculate perplexity based on the dev loss (assuming cross-entropy loss)
    dev_perplexity_mode2 = math.exp(dev_loss_mode2)
    dev_perplexity_mode2_values.append(dev_perplexity_mode2)

    print(f"Epoch {epoch+1}/{num_epochs} (Mode 2) - Dev Loss: {dev_loss_mode2:.4f}, Dev Perplexity: {dev_perplexity_mode2:.4f}")

# Testing loop
test_loss_mode1 = test_mode1(elmo, test_loader, criterion, device)
test_loss_mode2 = test_mode2(elmo, test_loader, criterion, device)

# Calculate perplexity based on the test loss (assuming cross-entropy loss)
test_perplexity_mode1 = math.exp(test_loss_mode1)
test_perplexity_mode2 = math.exp(test_loss_mode2)

# Store test results
test_loss_mode1_values.append(test_loss_mode1)
test_perplexity_mode1_values.append(test_perplexity_mode1)
test_loss_mode2_values.append(test_loss_mode2)
test_perplexity_mode2_values.append(test_perplexity_mode2)

print(f"Test Loss (Mode 1): {test_loss_mode1:.4f}, Test Perplexity (Mode 1): {test_perplexity_mode1:.4f}")
print(f"Test Loss (Mode 2): {test_loss_mode2:.4f}, Test Perplexity (Mode 2): {test_perplexity_mode2:.4f}")

# Plotting loss and perplexity graphs
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))

# Loss plot for Mode 1
plt.subplot(2, 2, 1)
plt.plot(range(num_epochs), loss_mode1_values, label="Train Loss (Mode 1)")
plt.plot(range(num_epochs), dev_loss_mode1_values, label="Dev Loss (Mode 1)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss (Mode 1)")
plt.legend()

# Perplexity plot for Mode 1
plt.subplot(2, 2, 2)
plt.plot(range(num_epochs), perplexity_mode1_values, label="Train Perplexity (Mode 1)")
plt.plot(range(num_epochs), dev_perplexity_mode1_values, label="Dev Perplexity (Mode 1)")
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.title("Perplexity (Mode 1)")
plt.legend()

# Loss plot for Mode 2
plt.subplot(2, 2, 3)
plt.plot(range(num_epochs), loss_mode2_values, label="Train Loss (Mode 2)")
plt.plot(range(num_epochs), dev_loss_mode2_values, label="Dev Loss (Mode 2)")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Loss (Mode 2)")
plt.legend()

# Perplexity plot for Mode 2
plt.subplot(2, 2, 4)
plt.plot(range(num_epochs), perplexity_mode2_values, label="Train Perplexity (Mode 2)")
plt.plot(range(num_epochs), dev_perplexity_mode2_values, label="Dev Perplexity (Mode 2)")
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.title("Perplexity (Mode 2)")
plt.legend()

plt.tight_layout()
plt.show()

# Display test results
print(f"Test Loss (Mode 1): {test_loss_mode1:.4f}, Test Perplexity (Mode 1): {test_perplexity_mode1:.4f}")
print(f"Test Loss (Mode 2): {test_loss_mode2:.4f}, Test Perplexity (Mode 2): {test_perplexity_mode2:.4f}")


# Experiment Below (with Top K Accuracy)

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from tqdm import tqdm

# # Define the loss function and optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(elmo.parameters(), lr=0.001)  # Adjust the learning rate as needed

# # Training loop for mode 1 (next word prediction)
# def train_mode1(model, train_loader, optimizer, criterion, device):
#     model.train()
#     total_loss = 0.0
#     total_correct = 0

#     for inputs in tqdm(train_loader):
#         inputs = inputs[0]
#         inputs = inputs.to(device)
#         optimizer.zero_grad()

#         # Use the first 5 words as input and the last 5 words as expected output
#         input_seq = inputs[:, :5]
#         target_seq = inputs[:, 1:]

#         # Forward pass
#         outputs = model(input_seq, mode=1)

#         # Calculate the values for the one-hot encoding
#         value_for_correct_idx = 0.1
#         value_for_other_indices = 0.9 / model.vocab_size

#         # Create the one-hot encoding tensor
#         target_one_hot = torch.full((target_seq.size(0), target_seq.size(1), model.vocab_size), value_for_other_indices).to(device)

#         # Set the correct indices to the specified value
#         target_one_hot.scatter_(2, target_seq.unsqueeze(2), value_for_correct_idx)

#         # Calculate the loss
#         loss = criterion(outputs, target_one_hot)
#         total_loss += loss.item()

#         # Calculate top-k accuracy
#         # predictions = outputs.view(inputs.size(0), inputs.size(1), model.vocab_size)
#         predictions = outputs
#         _, top_indices = torch.topk(predictions, 50, dim=2)  # Get the top-10 predicted indices
#         correct_predictions = torch.any(top_indices == target_seq.unsqueeze(2), dim=2).sum().item()
#         total_correct += correct_predictions

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#     accuracy = total_correct / (len(train_loader.dataset) * 5)  # Calculate accuracy based on the entire dataset
#     return total_loss / len(train_loader), accuracy

# # Training loop for mode 2 (previous word prediction)
# def train_mode2(model, train_loader, optimizer, criterion, device):
#     model.train()
#     total_loss = 0.0
#     total_correct = 0

#     for inputs in tqdm(train_loader):
#         inputs = inputs[0]
#         inputs = inputs.to(device)
#         optimizer.zero_grad()

#         # Use the first 5 words as input and the last 5 words as expected output
#         input_seq = inputs[:, :5]
#         target_seq = inputs[:, 1:]

#         # Forward pass
#         outputs = model(input_seq, mode=2)

#         # Calculate the values for the one-hot encoding
#         value_for_correct_idx = 0.5
#         value_for_other_indices = 0.5 / model.vocab_size

#         # Create the one-hot encoding tensor
#         target_one_hot = torch.full((target_seq.size(0), target_seq.size(1), model.vocab_size), value_for_other_indices).to(device)

#         # Set the correct indices to the specified value
#         target_one_hot.scatter_(2, target_seq.unsqueeze(2), value_for_correct_idx)

#         # Calculate the loss
#         loss = criterion(outputs, target_one_hot)
#         total_loss += loss.item()

#         # Calculate top-k accuracy
#         # predictions = outputs.view(inputs.size(0), inputs.size(1), model.vocab_size)
#         predictions = outputs
#         _, top_indices = torch.topk(predictions, 50, dim=2)  # Get the top-10 predicted indices
#         correct_predictions = torch.any(top_indices == target_seq.unsqueeze(2), dim=2).sum().item()
#         total_correct += correct_predictions

#         # Backward pass and optimization
#         loss.backward()
#         optimizer.step()

#     accuracy = total_correct / (len(train_loader.dataset) * 5)  # Calculate accuracy based on the entire dataset
#     return total_loss / len(train_loader), accuracy


In [ ]:
# # Set the device (CPU or GPU)
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# elmo.to(device)

# # Training hyperparameters
# num_epochs = 10  # Adjust as needed

# # Initialize empty lists to store accuracy values
# acc1_values = []
# acc2_values = []

# # Training loop for mode 1
# for epoch in range(num_epochs):
#     train_loss_mode1, acc1 = train_mode1(elmo, train_loader, optimizer, criterion, device)
#     acc1_values.append(acc1)
#     print(f"Epoch {epoch+1}/{num_epochs} (Mode 1) - Loss: {train_loss_mode1:.4f} , Accuracy = {acc1}")

# # Training loop for mode 2
# for epoch in range(num_epochs):
#     train_loss_mode2, acc2 = train_mode2(elmo, train_loader, optimizer, criterion, device)
#     acc2_values.append(acc2) 
#     print(f"Epoch {epoch+1}/{num_epochs} (Mode 2) - Loss: {train_loss_mode2:.4f} , Accuracy = {acc2}")

In [ ]:
# import matplotlib.pyplot as plt

# # Assuming you have recorded the accuracy values for each epoch in lists acc1 and acc2
# # acc1 contains accuracy values for Mode 1, and acc2 for Mode 2

# epochs = range(1, num_epochs+1)

# # Plot for Mode 1
# plt.figure(figsize=(10, 5))
# plt.plot(epochs, acc1_values, label='Mode 1', marker='o', linestyle='-')
# plt.title('Epoch vs. Accuracy (Mode 1)')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.grid(True)
# plt.show()

# # Plot for Mode 2
# plt.figure(figsize=(10, 5))
# plt.plot(epochs, acc2_values, label='Mode 2', marker='o', linestyle='-')
# plt.title('Epoch vs. Accuracy (Mode 2)')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()
# plt.grid(True)
# plt.show()

# Experiment Above

In [ ]:
# Convert n-grams to PyTorch tensors
train_data_elmo = torch.tensor(train_indices_elmo, dtype=torch.long)
dev_data_elmo = torch.tensor(dev_indices_elmo, dtype=torch.long)
test_data_elmo = torch.tensor(test_indices_elmo, dtype=torch.long)

# Combine data with labels
train_data_with_labels = TensorDataset(train_data_elmo[:10000], torch.tensor(train_labels[:10000], dtype=torch.long))
dev_data_with_labels = TensorDataset(dev_data_elmo[:500], torch.tensor(dev_labels[:500], dtype=torch.long))
test_data_with_labels = TensorDataset(test_data_elmo[:500], torch.tensor(test_labels[:500], dtype=torch.long))

# Create DataLoader instances
batch_size = 32

train_loader = DataLoader(train_data_with_labels, batch_size=batch_size, shuffle=False)
dev_loader = DataLoader(dev_data_with_labels, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data_with_labels, batch_size=batch_size, shuffle=False)

In [ ]:
# Display the first few sentences and labels for train, dev, and test sets in index form
print("Train Set Sentences (Indices):")
print(train_indices_elmo[:2])  # Displaying the first 2 sentences for brevity
print("Train Set Labels:")
print(train_labels[:2])

print("\nDevelopment Set Sentences (Indices):")
print(dev_indices_elmo[:2])
print("Development Set Labels:")
print(dev_labels[:2])

print("\nTest Set Sentences (Indices):")
print(test_indices_elmo[:2])
print("Test Set Labels:")
print(test_labels[:2])

In [ ]:
# Training loop for mode 0 (4-class classification) on GPU
def train_elmo(model, train_loader, optimizer, criterion, device):
    model.train()
    total_loss = 0.0
    total_correct = 0
    all_predicted = []
    all_actual = []

    for inputs, target_seq in tqdm(train_loader):
        inputs = inputs.to(device)
        target_seq = target_seq.to(device)
        optimizer.zero_grad()

        input_seq = inputs

        # Forward pass
        outputs = model(input_seq, mode=0)

        target_seq = target_seq - 1

        # Calculate the loss directly with integer class labels
        loss = criterion(outputs, target_seq)
        total_loss += loss.item()

        # Calculate correct predictions
        predicted_classes = torch.argmax(outputs, dim=1)
        correct_predictions = torch.sum(torch.eq(predicted_classes, target_seq))
        total_correct += correct_predictions.item()

        all_predicted.extend(predicted_classes.cpu().numpy())
        all_actual.extend(target_seq.cpu().numpy())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    accuracy = total_correct / len(all_actual)  # Calculate accuracy based on the entire dataset

    return total_loss / len(train_loader), accuracy, all_predicted, all_actual

# Evaluation loop for mode 0 (4-class classification) on GPU
def evaluate_elmo(model, dev_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    all_predicted = []
    all_actual = []

    with torch.no_grad():
        for inputs, target_seq in tqdm(dev_loader):
            inputs = inputs.to(device)
            target_seq = target_seq.to(device)

            input_seq = inputs

            # Forward pass
            outputs = model(input_seq, mode=0)

            target_seq = target_seq - 1

            # Calculate the loss directly with integer class labels
            loss = criterion(outputs, target_seq)
            total_loss += loss.item()

            # Calculate correct predictions
            predicted_classes = torch.argmax(outputs, dim=1)
            correct_predictions = torch.sum(torch.eq(predicted_classes, target_seq))
            total_correct += correct_predictions.item()

            all_predicted.extend(predicted_classes.cpu().numpy())
            all_actual.extend(target_seq.cpu().numpy())

    accuracy = total_correct / len(all_actual)  # Calculate accuracy based on the entire dataset

    return total_loss / len(dev_loader), accuracy, all_predicted, all_actual


# Test loop for mode 0 (4-class classification) on GPU
def evaluate_elmo(model, test_loader, criterion, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    all_predicted = []
    all_actual = []

    with torch.no_grad():
        for inputs, target_seq in tqdm(test_loader):
            inputs = inputs.to(device)
            target_seq = target_seq.to(device)

            input_seq = inputs

            # Forward pass
            outputs = model(input_seq, mode=0)

            target_seq = target_seq - 1

            # Calculate the loss directly with integer class labels
            loss = criterion(outputs, target_seq)
            total_loss += loss.item()

            # Calculate correct predictions
            predicted_classes = torch.argmax(outputs, dim=1)
            correct_predictions = torch.sum(torch.eq(predicted_classes, target_seq))
            total_correct += correct_predictions.item()

            all_predicted.extend(predicted_classes.cpu().numpy())
            all_actual.extend(target_seq.cpu().numpy())

    accuracy = total_correct / len(all_actual)  # Calculate accuracy based on the entire dataset

    return total_loss / len(test_loader), accuracy, all_predicted, all_actual

In [ ]:
# # Training hyperparameters
# num_epochs = 5

# # Initialize lists to store evaluation metrics
# accuracy_values = []
# precision_values = []
# recall_values = []
# f1_values = []

# # Initialize variables to store predicted and actual values
# all_predicted = []
# all_actual = []

# # Define the loss function and optimizer
# # criterion = nn.BCEWithLogitsLoss()
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(elmo.parameters(), lr=0.001)


# # Training loop for mode 0
# for epoch in range(num_epochs):
#     loss, accuracy, predicted, actual = train_elmo(elmo, train_loader, optimizer, criterion, device)

#     # Calculate precision, recall, and F1 score
#     precision = precision_score(actual, predicted, average='weighted')
#     recall = recall_score(actual, predicted, average='weighted')
#     f1 = f1_score(actual, predicted, average='weighted')

#     # Store evaluation metrics
#     accuracy_values.append(accuracy)
#     precision_values.append(precision)
#     recall_values.append(recall)
#     f1_values.append(f1)

#     # Store predicted and actual values for confusion matrix
#     all_predicted.extend(predicted)
#     all_actual.extend(actual)

#     print(f"Epoch {epoch+1}/{num_epochs} - Loss: {loss:.4f}, Accuracy: {accuracy:.2%}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

#     # # Plot evaluation metrics after each epoch
#     # plt.figure(figsize=(12, 6))

#     # # Accuracy plot
#     # plt.subplot(2, 2, 1)
#     # plt.plot(range(epoch + 1), accuracy_values, label="Accuracy")
#     # plt.xlabel("Epoch")
#     # plt.ylabel("Accuracy")
#     # plt.title("Accuracy")
#     # plt.legend()

#     # # Precision plot
#     # plt.subplot(2, 2, 2)
#     # plt.plot(range(epoch + 1), precision_values, label="Precision")
#     # plt.xlabel("Epoch")
#     # plt.ylabel("Precision")
#     # plt.title("Precision")
#     # plt.legend()

#     # # Recall plot
#     # plt.subplot(2, 2, 3)
#     # plt.plot(range(epoch + 1), recall_values, label="Recall")
#     # plt.xlabel("Epoch")
#     # plt.ylabel("Recall")
#     # plt.title("Recall")
#     # plt.legend()

#     # # F1 Score plot
#     # plt.subplot(2, 2, 4)
#     # plt.plot(range(epoch + 1), f1_values, label="F1 Score")
#     # plt.xlabel("Epoch")
#     # plt.ylabel("F1 Score")
#     # plt.title("F1 Score")
#     # plt.legend()

#     # plt.tight_layout()
#     # plt.show()

#     # # Replace this line with the correct number of classes in your confusion matrix
#     # num_classes = 4  # Change this to the actual number of classes

#     # # Display confusion matrix after each epoch
#     # conf_matrix = confusion_matrix(all_actual, all_predicted)
#     # plt.figure(figsize=(8, 6))
#     # plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
#     # plt.title('Confusion Matrix')
#     # plt.colorbar()
#     # tick_marks = np.arange(num_classes)  # Use the correct number of classes here
#     # plt.xticks(tick_marks, range(1, num_classes + 1), rotation=45)  # Adjust the labels as needed
#     # plt.yticks(tick_marks, range(1, num_classes + 1))  # Adjust the labels as needed
#     # plt.xlabel('Predicted')
#     # plt.ylabel('Actual')
#     # plt.show()


In [ ]:
# Training hyperparameters
num_epochs = 5

# Initialize lists to store evaluation metrics for training, dev, and test
train_accuracy_values = []
train_precision_values = []
train_recall_values = []
train_micro_f1_values = []

dev_accuracy_values = []
dev_precision_values = []
dev_recall_values = []
dev_micro_f1_values = []

test_accuracy_values = []
test_precision_values = []
test_recall_values = []
test_micro_f1_values = []

# Initialize variables to store predicted and actual values
train_all_predicted = []
train_all_actual = []

dev_all_predicted = []
dev_all_actual = []

test_all_predicted = []
test_all_actual = []

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(elmo.parameters(), lr=0.001)

# Training loop
for epoch in range(num_epochs):
    # Training
    train_loss, train_accuracy, train_predicted, train_actual = train_elmo(elmo, train_loader, optimizer, criterion, device)

    # Calculate precision, recall, and micro F1 score for training
    train_precision = precision_score(train_actual, train_predicted, average='weighted')
    train_recall = recall_score(train_actual, train_predicted, average='weighted')
    train_micro_f1 = f1_score(train_actual, train_predicted, average='micro')

    # Store training evaluation metrics
    train_accuracy_values.append(train_accuracy)
    train_precision_values.append(train_precision)
    train_recall_values.append(train_recall)
    train_micro_f1_values.append(train_micro_f1)

    train_all_predicted.extend(train_predicted)
    train_all_actual.extend(train_actual)

    print(f"Epoch {epoch+1}/{num_epochs} (Train) - Loss: {train_loss:.4f}, Accuracy: {train_accuracy:.2%}, Precision: {train_precision:.4f}, Recall: {train_recall:.4f}, Micro F1 Score: {train_micro_f1:.4f}")

    # Validation on dev_loader
    dev_loss, dev_accuracy, dev_predicted, dev_actual = evaluate_elmo(elmo, dev_loader, criterion, device)

    # Calculate precision, recall, and micro F1 score for dev_loader
    dev_precision = precision_score(dev_actual, dev_predicted, average='weighted')
    dev_recall = recall_score(dev_actual, dev_predicted, average='weighted')
    dev_micro_f1 = f1_score(dev_actual, dev_predicted, average='micro')

    # Store dev evaluation metrics per epoch
    dev_accuracy_values.append(dev_accuracy)
    dev_precision_values.append(dev_precision)
    dev_recall_values.append(dev_recall)
    dev_micro_f1_values.append(dev_micro_f1)

    dev_all_predicted.extend(dev_predicted)
    dev_all_actual.extend(dev_actual)

    print(f"Epoch {epoch+1}/{num_epochs} (Dev) - Loss: {dev_loss:.4f}, Accuracy: {dev_accuracy:.2%}, Precision: {dev_precision:.4f}, Recall: {dev_recall:.4f}, Micro F1 Score: {dev_micro_f1:.4f}")

# Testing on test_loader after training
test_loss, test_accuracy, test_predicted, test_actual = evaluate_elmo(elmo, test_loader, criterion, device)

# Calculate precision, recall, and micro F1 score for test_loader
test_precision = precision_score(test_actual, test_predicted, average='weighted')
test_recall = recall_score(test_actual, test_predicted, average='weighted')
test_micro_f1 = f1_score(test_actual, test_predicted, average='micro')

# Store test evaluation metrics
test_accuracy_values.append(test_accuracy)
test_precision_values.append(test_precision)
test_recall_values.append(test_recall)
test_micro_f1_values.append(test_micro_f1)

test_all_predicted.extend(test_predicted)
test_all_actual.extend(test_actual)

print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2%}, Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, Micro F1 Score: {test_micro_f1:.4f}")

# Plot evaluation metrics
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 18))

# Accuracy plot
plt.subplot(3, 2, 1)
plt.plot(range(num_epochs), train_accuracy_values, label="Train Accuracy")
plt.plot(range(num_epochs), dev_accuracy_values, label="Dev Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("Accuracy")
plt.legend()

# Precision plot
plt.subplot(3, 2, 2)
plt.plot(range(num_epochs), train_precision_values, label="Train Precision")
plt.plot(range(num_epochs), dev_precision_values, label="Dev Precision")
plt.xlabel("Epoch")
plt.ylabel("Precision")
plt.title("Precision")
plt.legend()

# Recall plot
plt.subplot(3, 2, 3)
plt.plot(range(num_epochs), train_recall_values, label="Train Recall")
plt.plot(range(num_epochs), dev_recall_values, label="Dev Recall")
plt.xlabel("Epoch")
plt.ylabel("Recall")
plt.title("Recall")
plt.legend()

# Micro F1 Score plot
plt.subplot(3, 2, 4)
plt.plot(range(num_epochs), train_micro_f1_values, label="Train Micro F1 Score")
plt.plot(range(num_epochs), dev_micro_f1_values, label="Dev Micro F1 Score")
plt.xlabel("Epoch")
plt.ylabel("Micro F1 Score")
plt.title("Micro F1 Score")
plt.legend()

# Confusion Matrix for Test Data
conf_matrix = confusion_matrix(test_all_actual, test_all_predicted)
plt.subplot(3, 2, 5)
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Test Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(conf_matrix))
plt.xticks(tick_marks, range(1, 5), rotation=45)
plt.yticks(tick_marks, range(1, 5))
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.show()